<a href="https://colab.research.google.com/github/trailblazerakash/MachineLearning/blob/master/hdfc/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import numpy as np
%matplotlib inline

from itertools import chain
from sklearn import preprocessing, ensemble
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split




In [0]:

import matplotlib.pyplot as plt
import seaborn as sns


from collections import Counter

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier, VotingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold, learning_curve

sns.set(style='white', context='notebook', palette='deep')

In [11]:

train_df = pd.read_csv('/content/drive/My Drive/HDFC/d407ebe0db9a11e9/DataSet/Train.csv')
test_df=pd.read_csv('/content/drive/My Drive/HDFC/d407ebe0db9a11e9/DataSet/Test.csv')


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (746,835) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (700,731,740,752,761,789,811,820,829,841,850) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
df_dtypes = pd.DataFrame(train_df.dtypes,columns=['dtypes'])
df_dtypes['first_value'] = train_df.loc[0].values


In [0]:
df_dtypes['Null_Count'] = train_df[train_df.columns].isnull().sum()
   

In [6]:
df_dtypes.head()

,dtypes,first_value,Null_Count
Col1,object,RIGHGSN,0
Col2,int64,1,0
Col3,float64,10867.1,0
Col4,float64,8648.34,0
Col5,float64,15766.2,0


In [7]:
df.isna().mean()

NameError: ignored

In [0]:
tdf = train_df.loc[:, train_df.isna().mean() < .25]
test_df_itm=test_df.loc[:, test_df.isna().mean() < .25]

In [0]:
test_df.shape

(20442, 2394)

In [0]:
def get_columns_count(df):
  dtype_df = df.dtypes.reset_index()
  dtype_df.columns = ["Count", "Column Type"]
  return dtype_df.groupby("Column Type").aggregate('count').reset_index()

In [14]:
get_columns_count(tdf)

,Column Type,Count
0,int64,1548
1,float64,566
2,object,1


In [15]:
get_columns_count(test_df)

,Column Type,Count
0,int64,1545
1,float64,837
2,object,12


In [9]:
dtype_df = tdf.dtypes.reset_index()
dtype_df.columns = ["Count", "Column Type"]
dtype_df.groupby("Column Type").aggregate('count').reset_index()


,Column Type,Count
0,int64,1548
1,float64,566
2,object,1


In [0]:
dtype_df.ix[:10,:]

In [0]:
unique_values_dict = {}
for col in tdf.columns:
    if col not in ["Col1","Col2"]:
        unique_value = str(np.sort(tdf[col].unique()).tolist())
        tlist = unique_values_dict.get(unique_value, [])
        tlist.append(col)
        unique_values_dict[unique_value] = tlist[:]
for unique_val, columns in unique_values_dict.items():
    print("Columns containing the unique values : ",unique_val)
    print(columns)
    print("--------------------------------------------------")

In [0]:
correlation_threshold = 0.99 #
train_integer = tdf.drop(["Col1","Col2"],axis=1)
cor = train_integer.corr()

In [17]:

cor.loc[:,:] =  np.tril(cor, k=-1)
cor = cor.stack()
correlated = cor[cor > correlation_threshold].reset_index().loc[:,['level_0','level_1']]
correlated = correlated.query('level_0 not in level_1')
correlated_array =  correlated.groupby('level_0').agg(lambda x: set(chain(x.level_0,x.level_1))).values
correlated_array

array([[{'Col946', 'Col1038'}],
       [{'Col1042', 'Col950'}],
       [{'Col982', 'Col1074'}],
       [{'Col1075', 'Col983'}],
       [{'Col108', 'Col94'}],
       [{'Col988', 'Col1080'}],
       [{'Col989', 'Col1081'}],
       [{'Col996', 'Col1088'}],
       [{'Col997', 'Col1089'}],
       [{'Col998', 'Col1090'}],
       [{'Col984', 'Col1168', 'Col1076'}],
       [{'Col1177', 'Col1085', 'Col993'}],
       [{'Col994', 'Col1086', 'Col1178'}],
       [{'Col1188', 'Col1096'}],
       [{'Col992', 'Col1084', 'Col1176', 'Col1268'}],
       [{'Col1288', 'Col1196', 'Col1104'}],
       [{'Col129', 'Col121'}],
       [{'Col1292', 'Col1200'}],
       [{'Col131', 'Col123'}],
       [{'Col1310', 'Col1326', 'Col1006'}],
       [{'Col125', 'Col133'}],
       [{'Col1022', 'Col1334'}],
       [{'Col1344', 'Col942'}],
       [{'Col1368', 'Col1352', 'Col962'}],
       [{'Col137', 'Col136'}],
       [{'Col1370', 'Col970'}],
       [{'Col978', 'Col1376'}],
       [{'Col1078', 'Col986', 'Col1392', 'Col1302

In [18]:
correlated_features = []
for sets in correlated_array:
    element_list = list(sets[0])
    for idx, el in enumerate(element_list):
        if idx is not 0:
            correlated_features.append(el)

print (correlated_features)


['Col1038', 'Col950', 'Col1074', 'Col983', 'Col94', 'Col1080', 'Col1081', 'Col1088', 'Col1089', 'Col1090', 'Col1168', 'Col1076', 'Col1085', 'Col993', 'Col1086', 'Col1178', 'Col1096', 'Col1084', 'Col1176', 'Col1268', 'Col1196', 'Col1104', 'Col121', 'Col1200', 'Col123', 'Col1326', 'Col1006', 'Col133', 'Col1334', 'Col942', 'Col1352', 'Col962', 'Col136', 'Col970', 'Col1376', 'Col986', 'Col1392', 'Col1302', 'Col1394', 'Col1416', 'Col1400', 'Col1106', 'Col1328', 'Col1114', 'Col1428', 'Col1342', 'Col1434', 'Col1458', 'Col1054', 'Col1460', 'Col1070', 'Col1482', 'Col1396', 'Col1306', 'Col1308', 'Col1398', 'Col1504', 'Col1506', 'Col1190', 'Col1508', 'Col1206', 'Col1430', 'Col1524', 'Col1548', 'Col1146', 'Col1154', 'Col1556', 'Col1262', 'Col1582', 'Col1406', 'Col1586', 'Col990', 'Col1410', 'Col1320', 'Col1082', 'Col1266', 'Col1590', 'Col1174', 'Col1502', 'Col1412', 'Col1322', 'Col1282', 'Col1580', 'Col1290', 'Col1330', 'Col1600', 'Col1420', 'Col1601', 'Col1421', 'Col1511', 'Col1298', 'Col161', 'C

In [0]:
non_cor_train_df = tdf.drop(correlated_features, axis=1 )

In [20]:
non_cor_train_df.shape


(17521, 1369)

In [21]:
non_cor_train_df.head()

,Col1,Col2,Col3,Col5,Col6,Col9,Col10,Col11,Col12,Col13,Col14,Col15,Col16,Col17,Col18,Col21,Col22,Col23,Col24,Col25,Col26,Col27,Col29,Col30,Col31,Col32,Col33,Col34,Col35,Col36,Col42,Col43,Col44,Col45,Col46,Col47,Col48,Col49,Col50,Col51,...,Col2324,Col2325,Col2329,Col2332,Col2334,Col2335,Col2336,Col2338,Col2340,Col2341,Col2342,Col2343,Col2344,Col2346,Col2347,Col2348,Col2350,Col2355,Col2356,Col2358,Col2359,Col2360,Col2361,Col2362,Col2366,Col2370,Col2371,Col2374,Col2375,Col2376,Col2378,Col2381,Col2382,Col2384,Col2385,Col2387,Col2389,Col2391,Col2392,Col2394
0,RIGHGSN,1,1.086711e+04,1.576618e+04,2.890466e+04,1.285597e+04,2.871509e+04,9.114586e+03,3.043078e+04,9.293496,8.570479,10.063117,10.271758,0.548538,0.403366,1.486524,1.728324,0.993441,1.297050,0.137398,0.115987,0.260428,1,1,1,1,1,1,0,6,6.07,6.07,2,3,2,3,2,2,2,2,...,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,214.0,68.0
1,RIMHI5ZGP,0,2.584312e+05,1.866287e+05,1.866287e+05,1.726538e+05,1.666974e+05,2.013518e+05,1.859404e+05,12.462385,12.280493,12.167132,12.136876,1.167898,2.676398,0.792123,0.934556,0.893203,0.966620,0.753892,0.808295,1.414214,1,1,3,4,4,0,4,5,116.65,116.65,3,0,2,0,2,0,1,0,...,1,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,41.0,38.0
2,RIHOG1SZU,0,1.459355e+07,1.100315e+07,1.010315e+07,5.320778e+06,6.906324e+06,9.368338e+06,1.113330e+07,16.496090,16.749305,16.532874,16.128357,1.145182,2.203874,0.422264,0.498347,0.683582,0.735339,0.808115,0.776910,1.414214,3,3,4,12,4,0,2,4,33.28,33.28,2,1,2,0,1,0,1,0,...,2,0,1,0,2,0,2,1,0,0,0,0,1,1,0,2,0,0,0,1,0,1,0,1,0,1,1,0,0,0,2,0,0,0,0,0,1,0,378.0,310.0
3,RIHOLMQWU,0,6.727043e+05,6.794893e+05,6.794893e+05,3.577134e+05,4.236612e+05,4.304848e+05,4.065539e+05,13.419061,13.624018,13.541837,13.429097,1.204010,0.536304,0.437243,0.521341,0.623499,0.534531,-0.125829,0.388830,-1.414214,2,2,2,24,1,0,1,4,-7.75,-7.75,2,1,2,1,1,1,1,1,...,1,0,2,0,2,0,3,1,0,0,1,2,2,0,1,0,1,0,0,0,0,0,0,0,0,1,2,0,0,0,2,0,0,1,2,0,0,0,702.0,644.0
4,RIHO584ET,0,-1.218742e+06,-1.666241e+05,-2.403574e+06,5.938358e+06,5.446532e+06,3.473444e+06,3.262168e+06,NaN,14.764533,14.551021,NaN,NaN,NaN,NaN,1.344637,NaN,1.563430,-0.113101,0.617486,1.414214,4,3,4,68,8,0,2,4,NaN,NaN,1,2,1,2,1,2,1,2,...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,1,0,0,0,0,0,0,0,0,0,0,0,1305.0,1211.0


In [0]:
import seaborn as sns; sns.set(style="ticks", color_codes=True)
g = sns.pairplot(non_cor_train_df)

In [22]:
unique_values_list = non_cor_train_df.nunique()
unique_values_list=unique_values_list.reset_index()
unique_values_list.columns = ['Col',"Count"]
unique_values_list['Col'].count()

1369

In [23]:
continious =  unique_values_list[unique_values_list['Count'] > 990]
cont_cols = continious['Col']
cont_df = non_cor_train_df[cont_cols]
cat_df = non_cor_train_df.drop(cont_cols,axis=1)
cat_df.head()

,Col2,Col29,Col30,Col31,Col32,Col33,Col34,Col35,Col36,Col44,Col45,Col46,Col47,Col48,Col49,Col50,Col51,Col52,Col53,Col54,Col55,Col56,Col57,Col58,Col59,Col60,Col61,Col62,Col63,Col64,Col65,Col66,Col67,Col68,Col69,Col70,Col71,Col72,Col73,Col74,...,Col2324,Col2325,Col2329,Col2332,Col2334,Col2335,Col2336,Col2338,Col2340,Col2341,Col2342,Col2343,Col2344,Col2346,Col2347,Col2348,Col2350,Col2355,Col2356,Col2358,Col2359,Col2360,Col2361,Col2362,Col2366,Col2370,Col2371,Col2374,Col2375,Col2376,Col2378,Col2381,Col2382,Col2384,Col2385,Col2387,Col2389,Col2391,Col2392,Col2394
0,1,1,1,1,1,1,1,0,6,2,3,2,3,2,2,2,2,1,3,1,3,1,2,1,2,1,1,1,1,1,1,1,1,3,1,3,1,3,1,3,...,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,214.0,68.0
1,0,1,1,3,4,4,0,4,5,3,0,2,0,2,0,1,0,3,0,2,0,2,0,1,0,2,0,2,0,2,0,1,0,2,0,2,0,2,0,2,...,1,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,41.0,38.0
2,0,3,3,4,12,4,0,2,4,2,1,2,0,1,0,1,0,2,1,2,0,1,0,1,0,1,1,1,0,1,0,1,0,2,1,2,1,1,0,1,...,2,0,1,0,2,0,2,1,0,0,0,0,1,1,0,2,0,0,0,1,0,1,0,1,0,1,1,0,0,0,2,0,0,0,0,0,1,0,378.0,310.0
3,0,2,2,2,24,1,0,1,4,2,1,2,1,1,1,1,1,2,1,2,1,1,1,1,1,2,0,2,0,1,0,1,0,2,1,2,1,2,1,2,...,1,0,2,0,2,0,3,1,0,0,1,2,2,0,1,0,1,0,0,0,0,0,0,0,0,1,2,0,0,0,2,0,0,1,2,0,0,0,702.0,644.0
4,0,4,3,4,68,8,0,2,4,1,2,1,2,1,2,1,2,1,2,1,2,1,2,1,2,0,2,0,2,0,2,0,2,1,2,1,2,1,2,1,...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,1,0,0,0,0,0,0,0,0,0,0,0,1305.0,1211.0


In [24]:
wrongly_classified =['Col2392','Col2394']
for col in wrongly_classified:
  cont_df[col] = non_cor_train_df[col]
  cat_df.drop(col,axis=1,inplace=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [0]:
cat_df = cat_df.drop('Col2', axis=1)

In [0]:
def get_missing_count(df):
  missing_df = df.isnull().sum(axis=0).reset_index()
  missing_df.columns = ['column_name', 'missing_count']
  missing_df = missing_df.loc[missing_df['missing_count']>0]
  return missing_df

In [0]:
def get_non_missing_columns(df):
  missing_df = df.isnull().sum(axis=0).reset_index()
  missing_df.columns = ['column_name', 'missing_count']
  missing_df = missing_df.loc[missing_df['missing_count']==0]
  return missing_df

In [0]:
cat_non_miss_columns =list(get_non_missing_columns(cat_df)['column_name'])
cont_non_miss_columns =list(get_non_missing_columns(cont_df)['column_name'])



In [29]:
final_cols = cat_non_miss_columns+cont_non_miss_columns

training_df = non_cor_train_df[final_cols]
training_df['Col2'] = non_cor_train_df['Col2']


#Expliciltly choosing only already decided features
testing_df = test_df_itm[final_cols]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [0]:
get_missing_count(testing_df)

,column_name,missing_count


In [30]:
ids= testing_df.Col1
testing_df.drop('Col1',axis=1,inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [0]:
target=training_df.Col2
candidate_train_predictors = training_df.drop(['Col1','Col2'], axis=1)
low_cardinality_cols = [cname for cname in candidate_train_predictors.columns
                                       if candidate_train_predictors[cname].nunique()< 20 and
                                       candidate_train_predictors[cname].dtype=="object"]

numeric_cols = [cname for cname in candidate_train_predictors.columns
                               if candidate_train_predictors[cname].dtype in ['int64', 'float64']]
useful_cols = low_cardinality_cols + numeric_cols
train_predictors = candidate_train_predictors[useful_cols]


In [0]:
one_hot_encoded_data = pd.get_dummies(train_predictors)

In [0]:
x_train,x_test,y_train,y_test = train_test_split(one_hot_encoded_data,target,test_size = 0.3,random_state = 1)

In [0]:
from sklearn.tree import DecisionTreeClassifier
dtree = DecisionTreeClassifier()
dtree.fit(x_train,y_train)
prediction = dtree.predict(x_test)

print('With Decision tree accuracy is: ',dtree.score(x_test,y_test))

With Decision tree accuracy is:  0.8333650370933993


In [0]:
prediction=dtree.predict(testing_df)


In [0]:
dataset = pd.DataFrame({'Col1': list(ids), 'Col2': prediction}, columns=['Col1', 'Col2'])
dataset.to_csv('/content/drive/My Drive/HDFC/d407ebe0db9a11e9/DataSet/Submission.csv',header=True,index=False)

In [0]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
param_grid = {"C": np.logspace(-3,3,7), "penalty" : ["l1","l2"]}

logreg = LogisticRegression()
logreg_cv = GridSearchCV(logreg,param_grid,cv = 10)
logreg_cv.fit(x_train,y_train)  

In [0]:
print("tuned hyperparameters: (best parameters): ", logreg_cv.best_params_)
print("accuracy: ", logreg_cv.best_score_)



In [0]:
logreg2 = LogisticRegression(C = 10, penalty="l2")
logreg2.fit(x_train,y_train)
print("score: ", logreg2.score(x_test,y_test))

In [0]:
xgboost = XGBClassifier(max_depth=5, learning_rate=0.01, n_estimators=100, gamma=0, 
                        min_child_weight=1, subsample=0.8, colsample_bytree=0.8, reg_alpha=0.005)

xgboost.fit(X_train, y_train)
preds = xgboost.predict(X_test)

accuracy = (preds == y_test).sum().astype(float) / len(preds)*100

print("XGBoost's prediction accuracy WITH optimal hyperparameters is: %3.2f" % (accuracy))

In [0]:


print("F1-Score:", f1_score(sdss['class'], predictions, average='micro'))



In [0]:
kfold = StratifiedKFold(n_splits=2)

In [40]:
SVMC = SVC(probability=True)
svc_param_grid = {'kernel': ['rbf'], 
                  'gamma': [ 0.001, 0.01, 0.1, 1],
                  'C': [1, 10, 50, 100,200,300, 1000]}

gsSVMC = GridSearchCV(SVMC,param_grid = svc_param_grid, cv=kfold, scoring="accuracy", n_jobs= 12, verbose = 1)

gsSVMC.fit(x_train,y_train)

SVMC_best = gsSVMC.best_estimator_

# Best score
gsSVMC.best_score_

Fitting 2 folds for each of 28 candidates, totalling 56 fits


[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.


KeyboardInterrupt: ignored

In [42]:
DTC = DecisionTreeClassifier()

adaDTC = AdaBoostClassifier(DTC, random_state=7)

ada_param_grid = {"base_estimator__criterion" : ["gini", "entropy"],
              "base_estimator__splitter" :   ["best", "random"],
              "algorithm" : ["SAMME","SAMME.R"],
              "n_estimators" :[1,2],
              "learning_rate":  [0.0001, 0.001, 0.01, 0.1, 0.2, 0.3,1.5]}

gsadaDTC = GridSearchCV(adaDTC,param_grid = ada_param_grid, cv=kfold, scoring="accuracy", n_jobs= 6, verbose = 4)

gsadaDTC.fit(x_train,y_train)

ada_best = gsadaDTC.best_estimator_

Fitting 2 folds for each of 112 candidates, totalling 224 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  13 tasks      | elapsed:   48.4s
[Parallel(n_jobs=6)]: Done  86 tasks      | elapsed:  3.0min
[Parallel(n_jobs=6)]: Done 209 tasks      | elapsed:  6.5min
[Parallel(n_jobs=6)]: Done 224 out of 224 | elapsed:  6.8min finished


In [44]:
ExtC = ExtraTreesClassifier()


## Search grid for optimal parameters
ex_param_grid = {"max_depth": [None],
              "max_features": [1, 3, 10],
              "min_samples_split": [2, 3, 10],
              "min_samples_leaf": [1, 3, 10],
              "bootstrap": [False],
              "n_estimators" :[100,300],
              "criterion": ["gini"]}


gsExtC = GridSearchCV(ExtC,param_grid = ex_param_grid, cv=kfold, scoring="accuracy", n_jobs= 6, verbose = 4)

gsExtC.fit(x_train,y_train)

ExtC_best = gsExtC.best_estimator_

# Best score
gsExtC.best_score_

Fitting 2 folds for each of 54 candidates, totalling 108 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=6)]: Done  13 tasks      | elapsed:   30.8s
[Parallel(n_jobs=6)]: Done  86 tasks      | elapsed:  2.5min
[Parallel(n_jobs=6)]: Done 108 out of 108 | elapsed:  3.3min finished


0.8990541422048272

In [45]:
RFC = RandomForestClassifier()


## Search grid for optimal parameters
rf_param_grid = {"max_depth": [None],
              "max_features": [1, 3, 10],
              "min_samples_split": [2, 3, 10],
              "min_samples_leaf": [1, 3, 10],
              "bootstrap": [False],
              "n_estimators" :[100,300],
              "criterion": ["gini"]}


gsRFC = GridSearchCV(RFC,param_grid = rf_param_grid, cv=kfold, scoring="accuracy", n_jobs= 4, verbose = 1)

gsRFC.fit(x_train,y_train)

RFC_best = gsRFC.best_estimator_

# Best score
gsRFC.best_score_

Fitting 2 folds for each of 54 candidates, totalling 108 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   58.8s
[Parallel(n_jobs=4)]: Done 108 out of 108 | elapsed:  3.5min finished


0.8990541422048272

In [47]:
GBC = GradientBoostingClassifier()
gb_param_grid = {'loss' : ["deviance"],
              'n_estimators' : [100,200,300],
              'learning_rate': [0.1, 0.05, 0.01],
              'max_depth': [4, 8],
              'min_samples_leaf': [100,150],
              'max_features': [0.3, 0.1] 
              }

gsGBC = GridSearchCV(GBC,param_grid = gb_param_grid, cv=kfold, scoring="accuracy", n_jobs= 6, verbose = 1)

gsGBC.fit(x_train,y_train)

GBC_best = gsGBC.best_estimator_

# Best score
gsGBC.best_score_

Fitting 2 folds for each of 72 candidates, totalling 144 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:  9.4min
[Parallel(n_jobs=6)]: Done 144 out of 144 | elapsed: 32.9min finished


0.8995433789954338

In [0]:
votingC = VotingClassifier(estimators=[('rfc', RFC_best), ('extc', ExtC_best),
 ('adac',ada_best),('gbc',GBC_best)], voting='soft', n_jobs=4)

votingC = votingC.fit(x_train, y_train)

In [0]:
prediction = votingC.predict(testing_df)
dataset = pd.DataFrame({'Col1': list(ids), 'Col2': prediction}, columns=['Col1', 'Col2'])
dataset.to_csv('/content/drive/My Drive/HDFC/d407ebe0db9a11e9/DataSet/Submission2.csv',header=True,index=False)